In [ ]:
import mne
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
channel_mapping = {
    'EEG FP1-REF': 'Fp1', 'EEG FP2-REF': 'Fp2',
    'EEG F3-REF': 'F3', 'EEG F4-REF': 'F4',
    'EEG C3-REF': 'C3', 'EEG C4-REF': 'C4',
    'EEG P3-REF': 'P3', 'EEG P4-REF': 'P4',
    'EEG O1-REF': 'O1', 'EEG O2-REF': 'O2',
    'EEG F7-REF': 'F7', 'EEG F8-REF': 'F8',
    'EEG T3-REF': 'T3', 'EEG T4-REF': 'T4',
    'EEG T5-REF': 'T5', 'EEG T6-REF': 'T6',
    'EEG FZ-REF': 'Fz', 'EEG CZ-REF': 'Cz',
    'EEG PZ-REF': 'Pz'
}

In [ ]:
def convertDF2MNE(sub, sfreq):
    """
    Convierte un DataFrame de datos EEG en un objeto MNE Epochs.

    Parameters:
    - sub (pd.DataFrame): DataFrame con los datos EEG.
    - sfreq (float): Frecuencia de muestreo de los datos.

    Returns:
    - numpy.ndarray: Datos de epochs en formato (n_epochs, n_channels, n_samples).
    """
    info = mne.create_info(list(sub.columns), ch_types=['eeg'] * len(sub.columns), sfreq=sfreq)
    info.set_montage('standard_1020')
    data = mne.io.RawArray(sub.T, info)
    data.set_eeg_reference()
    # data.filter(l_freq=1, h_freq=30)
    epochs = mne.make_fixed_length_epochs(data, duration=3, overlap=2)
    return epochs.get_data()

In [ ]:
def extract_dataframes(edf_file, annotations_file, artifact):
    """
    Extrae segmentos de datos EEG correspondientes a un tipo específico de anotación.

    Parameters:
    - edf_file (str): Ruta al archivo EDF.
    - annotations_file (str): Ruta al archivo de anotaciones CSV.
    - artifact (str): Tipo de anotación a extraer (e.g., 'musc').

    Returns:
    - numpy.ndarray: Datos de epochs en formato (n_epochs, n_channels, n_samples), o None si no se encuentra el artifact.
    """
    try:
        raw = mne.io.read_raw_edf(edf_file, preload=True, verbose='ERROR')
        
        channels_to_keep = [ch for ch in raw.ch_names if ch in channel_mapping]
        raw.pick(channels_to_keep)
        raw.rename_channels(channel_mapping)
        
        annotations_df = pd.read_csv(annotations_file, comment='#')
        
        annotation_mapping = {ch.split()[1]: new_name for ch, new_name in channel_mapping.items()}
        
        def map_annotation_channel(channel):
            try:
                ch1, ch2 = channel.split('-')
                mapped_ch1 = annotation_mapping.get(ch1, ch1)
                mapped_ch2 = annotation_mapping.get(ch2, ch2)
                return f"{mapped_ch1}-{mapped_ch2}"
            except ValueError:
                return channel
        
        if 'channel' not in annotations_df.columns:
            raise KeyError("La columna 'channel' no se encontró en el archivo de anotaciones.")
        
        annotations_df['channel'] = annotations_df['channel'].map(map_annotation_channel)
        
        valid_channels = [f"{ch1}-{ch2}" for ch1 in annotation_mapping.values() for ch2 in annotation_mapping.values()]
        annotations_df = annotations_df[annotations_df['channel'].isin(valid_channels)]
        
        annotations = mne.Annotations(
            onset=annotations_df['start_time'].values,
            duration=(annotations_df['stop_time'] - annotations_df['start_time']).values,
            description=annotations_df['label'].values
        )
        
        raw.set_annotations(annotations)
        
        target_annotations = raw.annotations[raw.annotations.description == artifact]
        
        if len(target_annotations) == 0:
            print(f"No se encontraron anotaciones para el artifact '{artifact}' en el archivo {edf_file}.")
            return None
        
        segments = []
        sfreq = raw.info['sfreq']
        
        for onset, duration in zip(target_annotations.onset, target_annotations.duration):
            tmin = onset
            tmax = onset + duration
            start_sample = int(tmin * sfreq)
            end_sample = int(tmax * sfreq)
            data, times = raw[:, start_sample:end_sample]
            df_segment = pd.DataFrame(data.T, columns=raw.ch_names)
            segments.append(df_segment)
        
        if not segments:
            print(f"No se encontraron segmentos para las anotaciones '{artifact}'.")
            return None
        
        df_artifact = pd.concat(segments, ignore_index=True)
        
        epochs_data = convertDF2MNE(df_artifact, sfreq)
        
        return epochs_data
    except Exception as e:
        print(f"Error procesando el archivo {edf_file} para el artifact '{artifact}': {e}")
        return None

In [ ]:
def find_matching_edf_csv_files(directory):
    edf_files = set()
    csv_files = set()

    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.edf'):
                edf_files.add(os.path.splitext(file)[0])
            elif file.endswith('.csv'):
                csv_files.add(os.path.splitext(file)[0])

    matching_files = edf_files.intersection(csv_files)

    matching_edf = [os.path.join(directory, f"{name}.edf") for name in matching_files]
    matching_csv = [os.path.join(directory, f"{name}.csv") for name in matching_files]

    return matching_edf, matching_csv

In [ ]:
def process_and_save_scaleograms(edf_file, annotations_file, artifacts, output_dir='./scaleogram'):
    """
    Procesa las anotaciones especificadas, aplica CWT usando ssqueezepy y guarda los resultados como archivos .npy.

    Parameters:
    - edf_file (str): Ruta al archivo EDF.
    - annotations_file (str): Ruta al archivo de anotaciones CSV.
    - artifacts (dict): Diccionario con tipos de anotaciones y sus etiquetas (e.g., {'eyem_musc': 0, 'musc_elec': 1}).
    - output_dir (str): Directorio base para guardar los archivos .npy.

    Returns:
    - pd.DataFrame: DataFrame con las rutas de los archivos .npy y sus etiquetas.
    """
    labels = []
    paths = []
    
    os.makedirs(output_dir, exist_ok=True)
    
    for artifact, label in artifacts.items():
        print(f"Procesando anotación: {artifact} con etiqueta: {label}")
        
        try:
            data = extract_dataframes(edf_file, annotations_file, artifact)
        except Exception as e:
            print(f"Error al procesar '{artifact}': {e}")
            continue
        
        for trial_idx, trial_data in enumerate(data):
            scaleograms = []
            
            for channel_idx, channel_data in enumerate(trial_data):
                Wx, scales = cwt(channel_data, 'morlet')
                Wx = np.abs(Wx)
                scaleograms.append(Wx)
            
            scaleograms = np.array(scaleograms)
            
            label_name = 'df_eyem_musc' if label == 0 else 'df_musc_elec'
            label_dir = os.path.join(output_dir, f'{label_name}')
            os.makedirs(label_dir, exist_ok=True)
            
            file_name = f'trial_{trial_idx}.npy'
            file_path = os.path.join(label_dir, file_name)
            
            np.save(file_path, scaleograms)
            
            paths.append(file_path)
            labels.append(label)
    
    df_scale = pd.DataFrame({'path': paths, 'label': labels})
    
    return df_scale

In [ ]:
directory = 'v3.0.1/edf/01_tcp_ar/'

matching_edf_files, matching_csv_files = find_matching_edf_csv_files(directory)

artifacts = {
    'musc': 0,
    'eyem_musc': 1,
    'elec': 2,
    'chew': 3
}

print("Archivos EDF y CSV coincidentes:")
for edf_file, csv_file in zip(matching_edf_files, matching_csv_files):
    print(f"EDF: {edf_file}, CSV: {csv_file}")
    
    df_musc = extract_dataframes(edf_file, csv_file, 'musc')
    if df_musc is not None:
        print(f"Forma del array 'df_musc': {df_musc.shape}")
    else:
        print(f"No se pudo extraer 'musc' del archivo {edf_file}.")
    
    df_eyem_musc = extract_dataframes(edf_file, csv_file, 'eyem_musc')
    if df_eyem_musc is not None:
        print(f"Forma del array 'df_eyem_musc': {df_eyem_musc.shape}")
    else:
        print(f"No se pudo extraer 'eyem_musc' del archivo {edf_file}.")

    df_elec = extract_dataframes(edf_file, csv_file, 'elec')
    if df_elec is not None:
        print(f"Forma del array 'df_elec': {df_elec.shape}")
    else:
        print(f"No se pudo extraer 'elec' del archivo {df_elec}.")

    df_chew = extract_dataframes(edf_file, csv_file, 'chew')
    if df_chew is not None:
        print(f"Forma del array 'df_chew': {df_chew.shape}")
    else:
        print(f"No se pudo extraer 'chew' del archivo {df_chew}.")

    df_scale = process_and_save_scaleograms(edf_file, annotations_file, artifacts, output_dir='./scaleogram_output')
    



In [2]:
import os
import mne
import pandas as pd
import numpy as np
from ssqueezepy import cwt
import matplotlib.pyplot as plt

# Mapeo de canales
channel_mapping = {
    'EEG FP1-REF': 'Fp1', 'EEG FP2-REF': 'Fp2',
    'EEG F3-REF': 'F3', 'EEG F4-REF': 'F4',
    'EEG C3-REF': 'C3', 'EEG C4-REF': 'C4',
    'EEG P3-REF': 'P3', 'EEG P4-REF': 'P4',
    'EEG O1-REF': 'O1', 'EEG O2-REF': 'O2',
    'EEG F7-REF': 'F7', 'EEG F8-REF': 'F8',
    'EEG T3-REF': 'T3', 'EEG T4-REF': 'T4',
    'EEG T5-REF': 'T5', 'EEG T6-REF': 'T6',
    'EEG FZ-REF': 'Fz', 'EEG CZ-REF': 'Cz',
    'EEG PZ-REF': 'Pz'
}

def convertDF2MNE(sub, sfreq):
    """
    Convierte un DataFrame de datos EEG en un objeto MNE Epochs.

    Parameters:
    - sub (pd.DataFrame): DataFrame con los datos EEG.
    - sfreq (float): Frecuencia de muestreo de los datos.

    Returns:
    - numpy.ndarray: Datos de epochs en formato (n_epochs, n_channels, n_samples).
    """
    info = mne.create_info(list(sub.columns), ch_types=['eeg'] * len(sub.columns), sfreq=sfreq)
    info.set_montage('standard_1020')
    data = mne.io.RawArray(sub.T, info)
    data.set_eeg_reference()
    # data.filter(l_freq=1, h_freq=30)  # Puedes descomentar para aplicar filtrado
    epochs = mne.make_fixed_length_epochs(data, duration=3, overlap=2)
    return epochs.get_data()

def extract_dataframes(edf_file, annotations_file, artifact):
    """
    Extrae segmentos de datos EEG correspondientes a un tipo específico de anotación.

    Parameters:
    - edf_file (str): Ruta al archivo EDF.
    - annotations_file (str): Ruta al archivo de anotaciones CSV.
    - artifact (str): Tipo de anotación a extraer (e.g., 'musc').

    Returns:
    - numpy.ndarray: Datos de epochs en formato (n_epochs, n_channels, n_samples), o None si no se encuentra el artifact.
    """
    try:
        # Cargar datos crudos
        raw = mne.io.read_raw_edf(edf_file, preload=True, verbose='ERROR')
        
        # Seleccionar y renombrar canales
        channels_to_keep = [ch for ch in raw.ch_names if ch in channel_mapping]
        raw.pick_channels(channels_to_keep)
        raw.rename_channels(channel_mapping)
        
        # Cargar anotaciones
        annotations_df = pd.read_csv(annotations_file, comment='#')
        
        # Crear mapeo de canales para anotaciones
        annotation_mapping = {ch.split()[1]: new_name for ch, new_name in channel_mapping.items()}
        
        # Definir la función de mapeo dentro del ámbito de la función
        def map_annotation_channel(channel):
            try:
                ch1, ch2 = channel.split('-')
                mapped_ch1 = annotation_mapping.get(ch1, ch1)
                mapped_ch2 = annotation_mapping.get(ch2, ch2)
                return f"{mapped_ch1}-{mapped_ch2}"
            except ValueError:
                return channel
        
        # Aplicar el mapeo a la columna 'channel'
        if 'channel' not in annotations_df.columns:
            raise KeyError("La columna 'channel' no se encontró en el archivo de anotaciones.")
        
        annotations_df['channel'] = annotations_df['channel'].map(map_annotation_channel)
        
        # Filtrar anotaciones válidas
        valid_channels = [f"{ch1}-{ch2}" for ch1 in annotation_mapping.values() for ch2 in annotation_mapping.values()]
        annotations_df = annotations_df[annotations_df['channel'].isin(valid_channels)]
        
        # Crear objeto Annotations de MNE
        annotations = mne.Annotations(
            onset=annotations_df['start_time'].values,
            duration=(annotations_df['stop_time'] - annotations_df['start_time']).values,
            description=annotations_df['label'].values
        )
        
        # Asignar anotaciones a los datos crudos
        raw.set_annotations(annotations)
        
        # Filtrar solo las anotaciones del tipo especificado
        target_annotations = raw.annotations[raw.annotations.description == artifact]
        
        if len(target_annotations) == 0:
            print(f"No se encontraron anotaciones para el artifact '{artifact}' en el archivo {edf_file}.")
            return None
        
        segments = []
        sfreq = raw.info['sfreq']
        
        for onset, duration in zip(target_annotations.onset, target_annotations.duration):
            tmin = onset
            tmax = onset + duration
            start_sample = int(tmin * sfreq)
            end_sample = int(tmax * sfreq)
            data, times = raw[:, start_sample:end_sample]
            df_segment = pd.DataFrame(data.T, columns=raw.ch_names)
            segments.append(df_segment)
        
        if not segments:
            print(f"No se encontraron segmentos para las anotaciones '{artifact}'.")
            return None
        
        # Concatenar todos los segmentos en un solo DataFrame
        df_artifact = pd.concat(segments, ignore_index=True)
        
        # Convertir el DataFrame a datos MNE Epochs
        epochs_data = convertDF2MNE(df_artifact, sfreq)
        
        return epochs_data
    except Exception as e:
        print(f"Error procesando el archivo {edf_file} para el artifact '{artifact}': {e}")
        return None

def find_matching_edf_csv_files(directory):
    """
    Encuentra archivos .edf y .csv que tengan el mismo nombre base en un directorio dado.

    Parameters:
    - directory (str): Ruta al directorio a buscar.

    Returns:
    - tuple: Listas de rutas completas para archivos .edf y .csv que coinciden.
    """
    edf_files = {}
    csv_files = {}

    # Recorre el directorio para identificar archivos .edf y .csv
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.edf'):
                name = os.path.splitext(file)[0]
                edf_files[name] = os.path.join(root, file)
            elif file.endswith('.csv'):
                name = os.path.splitext(file)[0]
                csv_files[name] = os.path.join(root, file)

    # Encuentra los nombres que están en ambos conjuntos
    matching_names = set(edf_files.keys()).intersection(csv_files.keys())

    # Crear listas de archivos .edf y .csv con los mismos nombres
    matching_edf = [edf_files[name] for name in matching_names]
    matching_csv = [csv_files[name] for name in matching_names]

    return matching_edf, matching_csv

def process_and_save_scaleograms(edf_file, annotations_file, artifacts, output_dir='./scaleogram_output', max_trials=100):
    """
    Procesa las anotaciones especificadas, aplica CWT usando ssqueezepy y guarda los resultados como archivos .npy.

    Parameters:
    - edf_file (str): Ruta al archivo EDF.
    - annotations_file (str): Ruta al archivo de anotaciones CSV.
    - artifacts (dict): Diccionario con tipos de anotaciones y sus etiquetas (e.g., {'eyem_musc': 0, 'musc_elec': 1}).
    - output_dir (str): Directorio base para guardar los archivos .npy.
    - max_trials (int): Número máximo de trials a generar por artefacto.

    Returns:
    - pd.DataFrame: DataFrame con las rutas de los archivos .npy y sus etiquetas.
    """
    labels = []
    paths = []
    
    # Asegurar que el directorio base exista
    os.makedirs(output_dir, exist_ok=True)
    
    for artifact, label in artifacts.items():
        print(f"Procesando anotación: {artifact} con etiqueta: {label}")
        
        # Extraer datos para la anotación actual
        try:
            data = extract_dataframes(edf_file, annotations_file, artifact)
        except Exception as e:
            print(f"Error al procesar '{artifact}': {e}")
            continue
        
        if data is None:
            print(f"No se pudo extraer datos para el artifact '{artifact}' en el archivo {edf_file}.")
            continue
        
        num_trials = data.shape[0]
        if num_trials > max_trials:
            print(f"Se encontraron {num_trials} trials para '{artifact}'. Seleccionando los primeros {max_trials}.")
            data = data[:max_trials]
        else:
            print(f"Se encontraron {num_trials} trials para '{artifact}'.")
        
        # Aplicar CWT y guardar archivos .npy
        for trial_idx, trial_data in enumerate(data):  # data tiene forma (trials, channels, samples)
            # Inicializar lista para almacenar los scaleograms de cada canal
            scaleograms = []
            
            for channel_idx, channel_data in enumerate(trial_data):
                # Aplicar CWT con la onda Morlet
                Wx, scales = cwt(channel_data, 'morlet')  # Puedes ajustar 'scales' si lo deseas
                Wx = np.abs(Wx)
                scaleograms.append(Wx)
            
            # Convertir la lista de scaleograms a un array 3D (channels, frequencies, time)
            scaleograms = np.array(scaleograms)  # Forma: (channels, frequencies, time)
            
            # Definir la ruta para guardar el archivo .npy
            label_dir = os.path.join(output_dir, f'label_{label}_{artifact}')
            os.makedirs(label_dir, exist_ok=True)
            
            # Nombre único para el archivo
            file_name = f'trial_{trial_idx}.npy'
            file_path = os.path.join(label_dir, file_name)
            
            # Guardar el scaleogram
            np.save(file_path, scaleograms)
            
            # Añadir la ruta y la etiqueta a las listas
            paths.append(file_path)
            labels.append(label)
        
        print(f"Se han guardado {min(num_trials, max_trials)} trials para el artifact '{artifact}'.\n")
    
    # Crear el DataFrame final
    df_scale = pd.DataFrame({'path': paths, 'label': labels})
    
    return df_scale

def main():
    # Ruta de la carpeta a buscar
    directory = 'v3.0.1/edf/01_tcp_ar/'
    
    # Obtener los archivos coincidentes
    matching_edf_files, matching_csv_files = find_matching_edf_csv_files(directory)
    
    if not matching_edf_files:
        print("No se encontraron archivos .edf y .csv coincidentes en el directorio especificado.")
        return
    
    # Definir los artefactos y sus etiquetas
    artifacts = {
        'musc': 0,
        'eyem_musc': 1,
        'elec': 2,
        'chew': 3
    }
    
    # Lista para almacenar los DataFrames de scaleograms de todos los archivos
    all_scale_dfs = []
    
    # Imprimir los resultados emparejados y extraer datos
    print("Archivos EDF y CSV coincidentes:")
    for edf_file, csv_file in zip(matching_edf_files, matching_csv_files):
        print(f"EDF: {edf_file}, CSV: {csv_file}")
        
        # Procesar y guardar los scaleograms
        df_scale = process_and_save_scaleograms(
            edf_file, 
            csv_file, 
            artifacts, 
            output_dir='./scaleogram_output', 
            max_trials=100  # Limitar a 100 trials por artefacto
        )
        
        if df_scale is not None and not df_scale.empty:
            all_scale_dfs.append(df_scale)
            # Mostrar las primeras filas del DataFrame resultante
            print(df_scale.head())
        else:
            print(f"No se generaron scaleograms para el archivo {edf_file}.\n")
    
    # Concatenar todos los DataFrames en uno solo
    if all_scale_dfs:
        final_df = pd.concat(all_scale_dfs, ignore_index=True)
        # Guardar el DataFrame final a un archivo CSV si lo deseas
        final_df.to_csv('scaleograms_labels.csv', index=False)
        print("Proceso completado. DataFrame final guardado en 'scaleograms_labels.csv'.")
    else:
        print("No se generaron scaleograms para ninguno de los archivos.")

if __name__ == "__main__":
    main()


Archivos EDF y CSV coincidentes:
EDF: v3.0.1/edf/01_tcp_ar/aaaaanrc_s004_t006.edf, CSV: v3.0.1/edf/01_tcp_ar/aaaaanrc_s004_t006.csv
Procesando anotación: musc con etiqueta: 0
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Creating RawArray with float64 data, n_channels=19, n_times=395444
    Range : 0 ... 395443 =      0.000 ...  1544.699 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Not setting metadata
1542 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1542 events and 768 original time points ...
0 bad epochs dropped
Se encontraron 1542 trials para 'musc'. Seleccionando los primeros 100.
Se han guardado 100 trials para el artifact 'musc'.

Procesando anotación: eyem_musc con etiqueta: 1
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
No se encontraron anotaciones para el arti

KeyboardInterrupt: 